In [1]:
import glob
import pandas as pd
import numpy as np
import pickle

In [2]:
data_path = "/Users/jurajkapasny/Data/sk_text_for_api"

In [11]:
def load_data(path):
    allFiles = glob.glob(path + "/*.csv")
    print ("Number of files to process: %d" %len(allFiles))
    df = pd.DataFrame()
    list_ = []
    i = 0
    for file_ in allFiles:
        if (file_.find("articles") != -1) & (file_.find("temo") == -1):
            i = i + 1
            if i%50 == 0:
                print ("Processing file %d" %i)
            temp = pd.read_csv(file_,sep = "|",index_col=None, header=0, parse_dates=True, low_memory=False)
            list_.append(temp)
    df = pd.concat(list_).reset_index(drop = True)
    return df

In [12]:
df = load_data(data_path)

Number of files to process: 211
Processing file 50
Processing file 100
Processing file 150
Processing file 200


In [13]:
df.head()

,label,summary,text
0,domov,Žilinčík zo Súdnej rady: Najhoršie veci robili...,\nDo Súdnej rady pribudol v novembri ako nomin...
1,korzar,Východ zasiahli silné búrky. Opäť mali charakt...,\nVÝCHODNÉ SLOVENSKO. Prehánky a búrky sa v so...
2,svet,Ruská polícia zadržala takmer stovku prívrženc...,\nMOSKVA. Ruská polícia zadržala desiatky stúp...
3,svet,"Turecko kvôli USA uvažuje, že neschváli dohodu...",\nHAMBURG. Rozhodnutie Spojených štátov odstúp...
4,ekonomika,Netflix pridal slovenské titulky k populárnym ...,\nBRATISLAVA. On-line videotéka Netflix začala...


In [14]:
df_clean = df[(df.label.notnull()) & (df.text.notnull())][["label","text"]]

In [15]:
df_clean.head()

,label,text
0,domov,\nDo Súdnej rady pribudol v novembri ako nomin...
1,korzar,\nVÝCHODNÉ SLOVENSKO. Prehánky a búrky sa v so...
2,svet,\nMOSKVA. Ruská polícia zadržala desiatky stúp...
3,svet,\nHAMBURG. Rozhodnutie Spojených štátov odstúp...
4,ekonomika,\nBRATISLAVA. On-line videotéka Netflix začala...


In [16]:
docs = df_clean.text.tolist()

In [18]:
docs[0]

"\nDo Súdnej rady pribudol v novembri ako nominant prezidenta Andreja Kisku a ako jeden z mála nesudcov, ktorí v 18-člennom zbore sedia. S väčšinovým názorom, že radu by mal viesť sudca, PAVOL ŽILINČÍK nesúhlasí. Vyberať by sa malo najmä podľa etických kritérií, prízvukuje právnik, ktorý pôsobil v organizácii Via Iuris, ako poradca na americkej ambasáde a dnes pracuje v kancelárii ombudsmanky.\nVidíte v čerstvo obmenenej Súdnej rade vhodného kandidáta na jej šéfa?\n„Pri voľbe predsedu by mali zohrať rolu dve okolnosti - či kandidát preukázal v minulosti dostatok integrity a tiež schopnosti zvládať ťažké situácie. Myslím si, že na prvom mieste mala byť práve otázka etiky a hodnotového zamerania. Na Slovensku to však zatiaľ nehrá prvé husle.“\nV Súdnej rade prevláda názor, že by ju mal viesť sudca, a to taký, ktorého do rady zvolili sudcovia. Stotožňujete sa s tým?\n„Nemyslím si, že to je nevyhnutná podmienka. Viem o krajinách, kde Súdne rady vedú nesudcovia, napríklad v Slovinsku. Zdá s

### Data Processing

In [19]:
from nltk.stem.wordnet import WordNetLemmatizer
import string

In [20]:
text_file = open("/Users/jurajkapasny/nltk_data/corpora/stopwords-sk/stopwords-sk.json", "r")
lines = text_file.read()
stopwords_sk = lines.replace('"','').split(",")

In [21]:
for i in range(len(stopwords_sk)):
    stopwords_sk[i] = stopwords_sk[i].replace("”","")
    stopwords_sk[i] = stopwords_sk[i].replace("“","")

In [24]:
exclude = set(string.punctuation) 
lemma = WordNetLemmatizer()
def clean(doc):
    stop_free = " ".join([i for i in doc.lower().split() if i not in stopwords_sk])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    num_free = ''.join(ch for ch in punc_free if ch not in ['0','1','2','3','4','5','6','7','8','9'])
    normalized = " ".join(lemma.lemmatize(word) for word in num_free.split())
    no_short = " ".join(word for word in normalized.split() if len(word) > 3)
    return no_short

In [25]:
doc_clean = [clean(doc).split() for doc in docs]  

In [27]:
for i in doc_clean[1]:
    print(i)

východné
slovensko
prehánky
búrky
sobotu
vyskytli
miestami
takmer
celom
území
východného
slovenska
informácií
slovenského
hydrometeorologického
ústavu
portálu
imeteosk
mali
búrky
popoludní
podvečer
vyskytli
regióne
charakter
supercely
popoludní
hodine
zasiahli
najmä
košický
kraj
miestami
veľmi
intenzívne
slovenský
hydrometeorologický
ústav
shmú
informuje
sociálnej
sieti
búrky
krupobitím
zaznamenali
popoludní
okrese
trebišov
zemplínsky
klečenov
prívalová
voda
zaliala
polia
niekde
napríklad
veľatoch
ulice
obcí
popoludní
okrese
trebišov
zdroj
imeteosk
silné
výdatné
popoludňajšie
búrky
zasiahli
spiš
oblasť
slovenského
raja
hnileckej
doliny
švedlári
napríklad
zaznamenali
minút
milimetrový
úhrn
zrážok
veľké
úhrny
zrážok
krátky
zaznamenali
okolí
košíc
gemeri
výrazná
búrková
činnosť
vrátila
okolia
košíc
východného
slovenska
opäť
večer
večer
košiciach
zdroj
imeteosk
neskôr
popoludní
búrky
presunuli
severnejšie
častejšie
vyskytovali
území
prešovského
kraja
podvečer
informácií
imeteosk
presunula


In [31]:
# Importing Gensim
import gensim
from gensim import corpora

In [32]:
# Creating the term dictionary of our courpus, where every unique term is assigned an index. 
dictionary = corpora.Dictionary(doc_clean)

In [33]:
# Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]

In [34]:
# Creating the object for LDA model using gensim library
lda = gensim.models.ldamodel.LdaModel

In [35]:
# Running and Trainign LDA model on the document term matrix.
ldamodel = lda(doc_term_matrix, num_topics=20, id2word = dictionary, passes=50)

In [36]:
pickle.dump( ldamodel, open( "/Users/jurajkapasny/Code/GitHub/jurajkapasny/text_analytics/LDA.p", "wb" ) )

In [37]:
ldamodel = pickle.load( open( "/Users/jurajkapasny/Code/GitHub/jurajkapasny/text_analytics/LDA.p", "rb" ) )

In [38]:
print ldamodel.print_topics(num_topics=20, num_words=10)[0][1]
print ""
print ldamodel.print_topics(num_topics=20, num_words=10)[1][1]
print ""
print ldamodel.print_topics(num_topics=20, num_words=10)[2][1]
print ""
print ldamodel.print_topics(num_topics=20, num_words=10)[3][1]
print ""
print ldamodel.print_topics(num_topics=20, num_words=10)[4][1]
print ""
print ldamodel.print_topics(num_topics=20, num_words=10)[5][1]
print ""
print ldamodel.print_topics(num_topics=20, num_words=10)[6][1]
print ""
print ldamodel.print_topics(num_topics=20, num_words=10)[7][1]
print ""
print ldamodel.print_topics(num_topics=20, num_words=10)[8][1]
print ""
print ldamodel.print_topics(num_topics=20, num_words=10)[9][1]
print ""
print ldamodel.print_topics(num_topics=20, num_words=10)[10][1]
print ""
print ldamodel.print_topics(num_topics=20, num_words=10)[11][1]
print ""
print ldamodel.print_topics(num_topics=20, num_words=10)[12][1]
print ""
print ldamodel.print_topics(num_topics=20, num_words=10)[13][1]
print ""
print ldamodel.print_topics(num_topics=20, num_words=10)[14][1]
print ""
print ldamodel.print_topics(num_topics=20, num_words=10)[15][1]
print ""
print ldamodel.print_topics(num_topics=20, num_words=10)[16][1]
print ""
print ldamodel.print_topics(num_topics=20, num_words=10)[17][1]
print "" 
print ldamodel.print_topics(num_topics=20, num_words=10)[18][1]
print "" 
print ldamodel.print_topics(num_topics=20, num_words=10)[19][1]
print "" 

SyntaxError: invalid syntax (<ipython-input-38-896ec181342d>, line 1)

In [51]:
print ldamodel.print_topics(num_topics=20, num_words=10)[19]
print "" 

(19, u'0.008*"valec" + 0.008*"kolies" + 0.006*"zadn\xfdch" + 0.005*"renault" + 0.004*"kombi" + 0.003*"koles\xe1" + 0.003*"golf" + 0.002*"nissan" + 0.002*"pneumatiky" + 0.002*"suzuki"')

